In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix
import matplotlib.mlab as mlab
%matplotlib inline


In [ ]:
heart_df=pd.read_csv("/content/Predict heart disease (Final).csv")
heart_df.head()

In [ ]:
heart_df.rename(columns={'Sex_male':'gender'},inplace=True)# here we change the col name from sex_male to gender
heart_df.head()

In [ ]:
heart_df.isnull().sum()# the data has cleand so there is no null values

In [ ]:
count=0
for i in heart_df.isnull().sum(axis=1):
    if i>0:
        count=count+1
print('Total number of rows with missing values is ', count)#there is no missing values (mean null values or empty cell)
print('since it is only',round((count/len(heart_df.index))*100), 'percent of the entire dataset the rows with missing values are excluded.')

In [ ]:
heart_df.dropna(axis=0,inplace=True)#dropna remove empty cell for cleaning
heart_df.head()

In [ ]:
def draw_histograms(dataframe, features, rows, cols):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(features):
        # print(dataframe) # dataframe display all the col and thier data and its 2dim array
        # print(features)# features display only the name of the col

        ax=fig.add_subplot(rows,cols,i+1)#15 col the row and cols mean how to display the chart
        dataframe[feature].hist(bins=20,ax=ax,facecolor='midnightblue')
        ax.set_title(feature+" Distribution",color='DarkRed')

    fig.tight_layout()
    plt.show()
draw_histograms(heart_df,heart_df.columns,6,3)

In [ ]:
heart_df.TenYearHD.value_counts()


In [ ]:
sn.countplot(x='TenYearHD',data=heart_df)#chart from seaborn to se how many pepole will have heart disease  in 10 years

In [ ]:
heart_df.describe()


In [ ]:
from statsmodels.tools import add_constant as add_constant
heart_df_constant = add_constant(heart_df)
heart_df_constant.head()

In [ ]:
st.chisqprob = lambda chisq, df: st.chi2.sf(chisq, df)
cols=heart_df_constant.columns[:-1]
model=sm.Logit(heart_df.TenYearHD,heart_df_constant[cols])
result=model.fit()
result.summary()

In [ ]:
def back_feature_elem (data_frame,dep_var,col_list):
    """ Takes in the dataframe, the dependent variable and a list of column names, runs the regression repeatedly eleminating feature with the highest
    P-value above alpha one at a time and returns the regression summary with all p-values below alpha"""

    while len(col_list)>0 :
        model=sm.Logit(dep_var,data_frame[col_list])
        result=model.fit(disp=0)
        largest_pvalue=round(result.pvalues,3).nlargest(1)
        if largest_pvalue[0]<(0.05):
            return result
            break
        else:
            col_list=col_list.drop(largest_pvalue.index)

result=back_feature_elem(heart_df_constant,heart_df.TenYearHD,cols)

In [ ]:
import sklearn
new_features=heart_df[['gender','age','cigsPerDay','totChol','sysBP','glucose','TenYearHD']]
x=new_features.iloc[:,:-1]
y=new_features.iloc[:,-1]
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.20,random_state=5)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression()
logreg.fit(x_train,y_train)
y_pred=logreg.predict(x_test)

In [ ]:
sklearn.metrics.accuracy_score(y_test,y_pred)


In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
conf_matrix=pd.DataFrame(data=cm,columns=['Predicted:0','Predicted:1'],index=['Actual:0','Actual:1'])
plt.figure(figsize = (8,5))
sn.heatmap(conf_matrix, annot=True,fmt='d',cmap="YlGnBu")

In [ ]:
TN=cm[0,0]
TP=cm[1,1]
FN=cm[1,0]
FP=cm[0,1]
sensitivity=TP/float(TP+FN)
specificity=TN/float(TN+FP)

In [ ]:
print('The acuuracy of the model = TP+TN/(TP+TN+FP+FN) = ',(TP+TN)/float(TP+TN+FP+FN),'\n',

'The Missclassification = 1-Accuracy = ',1-((TP+TN)/float(TP+TN+FP+FN)),'\n',

'Sensitivity or True Positive Rate = TP/(TP+FN) = ',TP/float(TP+FN),'\n',

'Specificity or True Negative Rate = TN/(TN+FP) = ',TN/float(TN+FP),'\n',

'Positive Predictive value = TP/(TP+FP) = ',TP/float(TP+FP),'\n',

'Negative predictive Value = TN/(TN+FN) = ',TN/float(TN+FN),'\n',

'Positive Likelihood Ratio = Sensitivity/(1-Specificity) = ',sensitivity/(1-specificity),'\n',

'Negative likelihood Ratio = (1-Sensitivity)/Specificity = ',(1-sensitivity)/specificity)

In [ ]:
y_pred_prob=logreg.predict_proba(x_test)[:,:]
y_pred_prob_df=pd.DataFrame(data=y_pred_prob, columns=['Prob of no heart disease (0)','Prob of Heart Disease (1)'])
y_pred_prob_df.head()

from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob_yes[:,1])
plt.plot(fpr,tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title('ROC curve for Heart disease classifier')
plt.xlabel('False positive rate (1-Specificity)')
plt.ylabel('True positive rate (Sensitivity)')
plt.grid(True)

Saving the trained model

In [ ]:
import pickle

In [ ]:
filename = 'heartDisease.sav'
pickle.dump(logreg, open(filename, 'wb'))

Test Saved Model

In [ ]:
# loading the saved model
loaded_model = pickle.load(open('heartDisease.sav', 'rb'))

In [ ]:
#There is no Problem.
#input_data = (1, 39, 0, 195, 106, 77)
#There is Problem.
#input_data = (0, 46, 20, 291, 112, 89)
#There is Problem.
#input_data = (0, 61, 30, 225, 150, 103)#
#There is Problem.
#input_data = (1, 39, 0, 285, 155, 70)

#There is Problem.
input_data = (1, 39, 70, 285, 155, 270)


# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = loaded_model.predict(input_data_reshaped)
print(prediction)

if (prediction[0] == 0):
  print('This person AAAAAAAAAAAAAAAA have no problem.')
else:
  print('This person AAAAAAAAAAAAAAAA have a problem.')

ONLY RUN When want to set predictions to a csv file.

In [ ]:
import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('heartDisease.csv')

# Assuming 'loaded_model' is your trained model
for index, row in df.iterrows():
    # Extract the features from the current row
    input_data = (row['gender'], row['age'], row['cigsPerDay'], row['totChol'], row['sysBP'], row['glucose'])

    # Convert input data to numpy array and reshape
    input_data_as_numpy_array = np.asarray(input_data)
    input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)

    # Make prediction
    prediction = loaded_model.predict(input_data_reshaped)

    # Add prediction to a new column in the DataFrame
    df.loc[index, 'prediction'] = np.where(prediction == 0, '0', '1')

# Save the updated DataFrame to the CSV file
df.to_csv('heart disease.csv', index=False)

# Print the updated DataFrame
print(df)


API for the Model

In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio
!pip install ngrok

API Generation

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
from http.server import HTTPServer, BaseHTTPRequestHandler
import logging, ngrok


In [ ]:
app = FastAPI()

origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
class model_input(BaseModel):

    Gender : int
    Age : int
    CigsPerDay : int
    TotChol : float
    SysBP : float
    Glucose : float

In [ ]:
# loading the saved model
diabetes_model = pickle.load(open('heartDisease.sav', 'rb'))

In [ ]:
@app.post('/Heart_PR')
def diabetes_predd(input_parameters : model_input):

    input_data = input_parameters.json()
    input_dictionary = json.loads(input_data)

    gender = input_dictionary['Gender']
    age = input_dictionary['Age']
    cigsPerDay = input_dictionary['CigsPerDay']
    totChol = input_dictionary['TotChol']
    sysBP = input_dictionary['SysBP']
    glucose = input_dictionary['Glucose']


    input_list = [gender, age, cigsPerDay, totChol, sysBP, glucose]

    predictionnnn = diabetes_model.predict([input_list])

    if (predictionnnn[0] == 0):
        return 'This person has no problem.'
    else:
        return 'This person has a problem.'

In [ ]:
from pyngrok import conf, ngrok

conf.get_default().auth_token = "Add Your Token For NGROK Channel"

In [ ]:
# Connect the tunnel
ngrok_tunnel = ngrok.connect(80)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=80)

Public URL: https://73e2-35-194-65-44.ngrok-free.app


INFO:     Started server process [595]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:80 (Press CTRL+C to quit)


INFO:     2a02:cb80:4135:a42f:fd4b:568c:ace8:6c21:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     51.211.186.12:0 - "POST /Heart_PR HTTP/1.1" 200 OK


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


INFO:     51.211.186.12:0 - "POST /Heart_PR HTTP/1.1" 200 OK


...

INFO:     51.211.186.12:0 - "POST /Heart_PR HTTP/1.1" 200 OK


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


INFO:     2a02:cb80:4135:a42f:fd4b:568c:ace8:6c21:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2a02:cb80:4135:a42f:fd4b:568c:ace8:6c21:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2a02:cb80:4135:a42f:fd4b:568c:ace8:6c21:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2a02:cb80:4135:a42f:fd4b:568c:ace8:6c21:0 - "GET / HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [595]


KeyboardInterrupt: 

In [ ]:
# Disconnect the tunnel using the public URL
ngrok_tunnel = ngrok.disconnect(ngrok_tunnel.public_url)

AttributeError: 'NoneType' object has no attribute 'public_url'